In [2]:
!git clone https://github.com/Nneji123/DeOldify.git

fatal: destination path 'DeOldify' already exists and is not an empty directory.


In [3]:
!cd DeOldify/

In [4]:
import os
os.chdir("/content/DeOldify/")
!pip install -r colab_requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 32.7 MB/s 
     |████████████████████████████████| 129 kB 48.7 MB/s 
     |████████████████████████████████| 1.9 MB 58.9 MB/s 
     |████████████████████████████████| 2.9 MB 64.4 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 86.3 MB 41 kB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 449 kB 65.1 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 3.8 MB 61.2 MB/s 
     |████████████████████████████████| 11.1 MB 60.1 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 80 k

In [5]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!ngrok authtoken 29vr8YhWZ9CDHrUq2kr0CpUA0e8_6ik4hU5GjJZeAYagTH5i4

--2022-06-15 20:37:56--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7247863 (6.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   6.91M  1.37MB/s    in 10s     

2022-06-15 20:38:07 (684 KB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [7247863/7247863]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
import os
os.chdir("/content/DeOldify/")
import uvicorn
from fastapi import FastAPI, File, UploadFile, Response
from fastapi.responses import StreamingResponse, FileResponse
import numpy as np
import io
from PIL import Image
import cv2
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)
import torch
import fastai
from ISR.models import RDN
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

app = FastAPI()

rdn = RDN(weights='noise-cancel')

colorizer = get_image_colorizer(artistic=True)


@app.get('/')
def home():
    return {'Title': 'Super Resolution and Colorisation API'}


# Endpoint for enhancing resolution and colorization of image
@app.post("/enhance_and_colorise")
async def root(file: UploadFile = File(...)):

    
    contents = io.BytesIO(await file.read())
    
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR) 
    
    cv2.imwrite("new.jpg",img)
    
    sr_img = rdn.predict(img, by_patch_of_size=300)
    
    res, im_png = cv2.imencode(".png", sr_img)
    
    images = io.BytesIO(im_png.tobytes())

    sr2_img = colorizer.get_transformed_image(images, render_factor=35)
    
    new_img = np.array(sr2_img)
    # img = cv2.imdecode(np.array(sr_img), cv2.IMREAD_COLOR)
    
    res, im2_png = cv2.imencode(".png", new_img) 

    return StreamingResponse(io.BytesIO(im2_png.tobytes()), media_type="image/png")


# endpoint for just enhancing the image
@app.post("/enhance")
async def root(file: UploadFile = File(...)):

    # image = load_image_into_numpy_array(await file.read())

    contents = io.BytesIO(await file.read())
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    sr_img = rdn.predict(img, by_patch_of_size=300)

    res, im_png = cv2.imencode(".png", sr_img)

    return StreamingResponse(io.BytesIO(im_png.tobytes()), media_type="image/png")

# endpoint for just colorising the image
# Please review this section I may have mixed up something
@app.post("/colorise")
async def root(file: UploadFile = File(...)):

    
    contents = io.BytesIO(await file.read())
    
    file_bytes = np.asarray(bytearray(contents.read()), dtype=np.uint8)
    
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR) 
    
    cv2.imwrite("new.jpg",img)
    
    #sr_img = rdn.predict(img, by_patch_of_size=300)
    
   # res, im_png = cv2.imencode(".png", sr_img)
    
    #images = io.BytesIO(im_png.tobytes())

    sr2_img = colorizer.get_transformed_image("new.jpg", render_factor=35)
    
    new_img = np.array(sr2_img)
    # img = cv2.imdecode(np.array(sr_img), cv2.IMREAD_COLOR)
    
    res, im2_png = cv2.imencode(".png", new_img) 

    return StreamingResponse(io.BytesIO(im2_png.tobytes()), media_type="image/png")







66076672/66071288 [==============================] - 6s 0us/step


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [9]:
from colabcode import ColabCode
cc = ColabCode(port=18000, code=False)
cc.run_app(app=app)

Public URL: NgrokTunnel: "http://de6a-35-247-175-187.ngrok.io" -> "http://localhost:18000"


In [8]:
!git config --global user.email "ifeanyinneji777@gmail.com"